In [11]:
import cv2
from tqdm import tqdm

import numpy as np

In [12]:
video_path = R"..\..\res\pont-cour.mp4"
pixels_to_meters = 0.02  # 1 pixel = 2 cm = 0.02 m a fine tuner
fps = 30                 # images/seconde
frame_skip = 1           # utiliser 1 = toutes les frames, 2 = une sur deux, etc.
max_frames = 100         # pour test rapide, ajuste à None pour toutes les frames
min_flow, max_flow = 0.5, 20  # seuils pour filtrer le bruit (pixels/frame)

In [13]:
cap = cv2.VideoCapture(video_path)
delta_t = 1 / fps
ret, prev = cap.read()
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
frame_count = 0
v_list = []
frame_idx = 0

In [14]:
with tqdm() as pbar:
    while True:
        for _ in range(frame_skip):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None,
                                            0.5, 3, 15, 3, 5, 1.2, 0)

        vx, vy = flow[..., 0], flow[..., 1]
        v = np.sqrt(vx**2 + vy**2)
        v_mean_pix = np.mean(v)
        v_surface = v_mean_pix * pixels_to_meters / delta_t

        v_list.append((frame_idx, v_surface))

        prev_gray = gray.copy()
        frame_idx += 1
        if max_frames and frame_idx >= max_frames:
            break
        pbar.update(1)

cap.release()

99it [01:01,  1.61it/s]


In [15]:
import csv
with open(R".\artifact\vitesse.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Frame", "Vitesse_surface_m/s"])
    writer.writerows(v_list)
